In [1]:
import pandas as pd
from pathlib import Path
import os
import joblib
import numpy as np
from mapie.metrics import (
    classification_coverage_score,
    classification_mean_width_score
)
import re
import matplotlib.pyplot as plt
from utils.model_production_data_processing_utils import cluster_with_min_size
import umap

root = Path(os.getcwd()).parent

In [2]:
from model_production_main import load_and_preprocess_data, prepare_features
from utils.model_production_data_processing_utils import build_X_s, build_umap_windows_by_suffix

In [3]:
from sklearn.cluster import KMeans


def compute_threshold_kmeans(
    df: pd.DataFrame,
    *,
    n_rendus: int = 3,
    min_prop: float = 0.5,
    max_prop: float = 1.0,
    n_grid: int = 100,
    min_dropout_pct: float = 20.0,
    random_state: int = 0
) -> float:
    """
    Calcule un seuil (threshold) à partir des dernières notes non nulles via un KMeans (k=2),
    puis balaye une grille de proportions pour maximiser la variation de la proportion binaire
    entre deux pas successifs.

    Paramètres
    ----------
    df : DataFrame
        Doit contenir des colonnes se terminant par 'mark'.
    n_rendus : int
        Nombre de notes non nulles les plus récentes à considérer (on prend la plus récente parmi elles).
    min_prop, max_prop : float
        Plage des coefficients appliqués au plus petit centre KMeans (min(centers)).
    n_grid : int
        Taille de la grille linéaire entre min_prop et max_prop.
    min_dropout_pct : float
        Si la proportion de Y_TARGET (dropout) < ce pourcentage, on recule d’un cran (prop courante).
    random_state : int
        Graine pour KMeans.

    Retour
    ------
    float
        Le threshold choisi.
    """

    # 1) Préparation des données
    df0 = df.copy().fillna(0)
    mark_cols = [c for c in df0.columns if c.endswith("mark")]
    if not mark_cols:
        raise ValueError("Aucune colonne se terminant par 'mark' trouvée.")

    # On inverse pour parcourir des colonnes de la plus récente à la plus ancienne (si votre ordre est encodé dans les noms)
    mark_cols = mark_cols[::-1]

    # 2) Récupérer pour chaque ligne la/les dernière(s) notes non nulles
    def last_marks(row):
        vals = []
        for c in mark_cols:
            v = row[c]
            if v > 0:
                vals.append(v)
                if len(vals) == n_rendus:
                    break
        return pd.Series({"last_vals": vals})

    tmp = df0.apply(last_marks, axis=1)
    df0["last_vals"] = tmp["last_vals"]
    # on prend la plus récente parmi les n_rendus collectés (index 0), sinon 0 si aucun
    df0["lastvals"] = df0["last_vals"].apply(lambda vs: vs[0] if len(vs) > 0 else 0.0).astype(float)

    x = df0["lastvals"].to_numpy(dtype=float)

    # 3) KMeans en 2 clusters
    X = x.reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=random_state).fit(X)
    centers = kmeans.cluster_centers_.flatten()
    low_center = float(np.min(centers))

    # 4) Balayage de la grille et sélection du meilleur "saut" de proportion
    props = np.linspace(min_prop, max_prop, n_grid)
    best_ec = -np.inf
    best_prop_prev = None
    best_prop_curr = None

    prev_mean = None
    for i, prop in enumerate(props):
        threshold_tmp = low_center * prop
        y_all = (x < threshold_tmp).astype(int)
        current_mean = float(y_all.mean())

        if i > 0:  # on peut mesurer l'écart avec l'itération précédente
            ec = abs(current_mean - prev_mean)
            if ec > best_ec:
                best_ec = ec
                best_prop_curr = prop           # prop à l'instant i
                best_prop_prev = props[i - 1]   # prop à l’instant i-1
        prev_mean = current_mean

    # Sécurités si la grille a une seule valeur ou si rien n'a été mis à jour
    if best_prop_prev is None or best_prop_curr is None:
        # fallback : prendre la première prop
        best_prop_prev = props[0]
        best_prop_curr = props[min(1, len(props) - 1)]

    # 5) Choix final du threshold + contrainte min_dropout_pct
    threshold = low_center * best_prop_prev
    y_target = (x < threshold).astype(int)
    if (y_target.mean() * 100.0) < min_dropout_pct:
        threshold = low_center * best_prop_curr
    y_target = (x < threshold).astype(int)
    print(y_target.mean() * 100.0)
    return float(threshold), (y_target.mean() * 100.0)


In [77]:
df22 = pd.read_csv(root /"data/DATA_2022.csv")
df22.shape

(1400, 401)

In [78]:
df23 = pd.read_csv(root /"data/DATA_2023.csv")
df23.shape

(1320, 486)

In [79]:
df24 = pd.read_csv(root / "data/DATA_2024.csv")
df24.shape

(851, 514)

In [4]:
df25 = pd.read_csv(root / "data/DATA_2025.csv")
df25.shape

(526, 25)

In [5]:
df25

,email,G-CPE-100_cpoolday01_01 - task01_passed,G-CPE-100_cpoolday01_02 - task02_passed,G-CPE-100_cpoolday01_03 - task03_passed,G-CPE-100_cpoolday01_04 - task04_passed,G-CPE-100_cpoolday01_05 - task05_passed,G-CPE-100_cpoolday01_06 - task06_passed,G-CPE-100_cpoolday01_07 - task07_passed,G-CPE-100_cpoolday01_08 - task08_passed,G-CPE-100_cpoolday01_09 - Success of task01_passed,...,G-CPE-100_cpoolday01_15 - Success of task07_passed,G-CPE-100_cpoolday01_16 - Success of task08_passed,G-CPE-100_cpoolday01_mark,G-CPE-100_cpoolday01_virtualMark,G-CPE-100_cpoolday01_prerequisitesMark,G-CPE-100_cpoolday01_stylePenalty,G-CPE-100_cpoolday01_styleFatal,G-CPE-100_cpoolday01_styleMajor,G-CPE-100_cpoolday01_styleMinor,G-CPE-100_cpoolday01_styleInfo
0,mohamed.moukhtari@epitech.eu,1,1,1,1,1,1,0,0,1,...,0,0,12.0,12.0,2.0,0,0,0,0,0
1,gregoire.riviere@epitech.eu,1,1,1,1,1,1,0,0,1,...,0,0,12.0,12.0,2.0,0,0,0,0,0
2,elias.rambaud@epitech.eu,1,1,1,1,1,1,0,0,1,...,0,0,12.0,12.0,2.0,0,0,0,0,0
3,ronan.richez@epitech.eu,1,1,1,1,0,0,0,0,1,...,0,0,7.0,7.0,2.0,0,0,0,0,0
4,louann.claro@epitech.eu,1,1,1,1,1,1,1,1,1,...,1,1,20.0,20.0,2.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,arthur.braga@epitech.eu,1,1,1,1,1,1,0,0,1,...,0,0,12.0,12.0,2.0,0,0,0,0,0
522,mathieu.guiton@epitech.eu,1,1,1,1,0,0,0,0,1,...,0,0,7.0,7.0,2.0,0,0,0,0,0
523,leo.casas-martinez@epitech.eu,0,0,0,0,0,0,0,0,0,...,0,0,1.0,2.0,1.0,0,0,0,0,0
524,marvin.ryan@epitech.eu,1,1,1,1,1,1,0,0,1,...,0,0,12.0,12.0,2.0,0,0,0,0,0


In [114]:
df2022 = load_and_preprocess_data(root / "data/DATA_2022.csv", 22)
threshold = compute_threshold_kmeans(df2022)
threshold

Data preprocessing done
41.785714285714285


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


(0.6114791621655578, 41.785714285714285)

In [112]:
df2023 = load_and_preprocess_data(root / "data/DATA_2023.csv", 23)
threshold = compute_threshold_kmeans(df2023)
threshold

Data preprocessing done
25.62396006655574


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


(0.3115646900783671, 25.62396006655574)

In [115]:
df2024 = load_and_preprocess_data(root / "data/DATA_2024.csv", 24)
threshold = compute_threshold_kmeans(df2024)
threshold

Data preprocessing done
24.941176470588236


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


(0.24260891833243772, 24.941176470588236)

In [116]:
common_cols = df22.columns.intersection(df23.columns).intersection(df24.columns)

# Concat verticale sur les colonnes communes
df_all = pd.concat(
    [df22[common_cols], df23[common_cols], df24[common_cols]],
    axis=0,
    ignore_index=True
)

mark_cols = [c for c in df_all.columns if c.endswith("mark")]
df_all[mark_cols] = df_all[mark_cols].div(df_all[mark_cols].mean())
threshold = compute_threshold_kmeans(df_all)
threshold

37.524502940352846


(0.4559874809686823, 37.524502940352846)

In [84]:
df_all 

,email,B-CPE-100_cpoolday01_01 - task01_passed,B-CPE-100_cpoolday01_02 - task02_passed,B-CPE-100_cpoolday01_03 - task03_passed,B-CPE-100_cpoolday01_04 - task04_passed,B-CPE-100_cpoolday01_05 - task05_passed,B-CPE-100_cpoolday01_06 - task06_passed,B-CPE-100_cpoolday01_07 - task07_passed,B-CPE-100_cpoolday01_mark,B-CPE-100_cpoolday01_virtualMark,...,B-PSU-200_42sh_22 - if_passed,B-PSU-200_42sh_23 - repeat_passed,B-PSU-200_42sh_mark,B-PSU-200_42sh_virtualMark,B-PSU-200_42sh_prerequisitesMark,B-PSU-200_42sh_stylePenalty,B-PSU-200_42sh_styleFatal,B-PSU-200_42sh_styleMajor,B-PSU-200_42sh_styleMinor,B-PSU-200_42sh_styleInfo
0,aaron.busnel@epitech.eu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085229,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaron.evers@epitech.eu,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.704581,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abdallah.hammad@epitech.eu,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.340916,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abdelhakim.tekaya@epitech.eu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085229,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelrahman.elwakil@epitech.eu,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.022749,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,zhamilya.kozhagulova@epitech.eu,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.534123,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3567,zhantore.svanov@epitech.eu,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.534123,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3568,zian.ferrage@epitech.eu,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.022749,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3569,zie-ange-mohamed.diawara@epitech.eu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.085229,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df24 = pd.read_csv(root / "data/DATA_2024.csv")
df24.shape

(851, 514)

In [86]:
[c for c in df22.columns if c.startswith('B-CPE-110')]

[]

In [87]:
cols_only_df1 = df23.columns.difference(df22.columns)
cols_only_df1

Index(['B-CPE-110_organized_Algorithm app_passed',
       'B-CPE-110_organized_Basics_passed',
       'B-CPE-110_organized_Optimization_passed',
       'B-CPE-110_organized_Parsing_passed',
       'B-CPE-110_organized_Robustness_passed', 'B-CPE-110_organized_mark',
       'B-CPE-110_organized_prerequisitesMark',
       'B-CPE-110_organized_styleFatal', 'B-CPE-110_organized_styleInfo',
       'B-CPE-110_organized_styleMajor', 'B-CPE-110_organized_styleMinor',
       'B-CPE-110_organized_stylePenalty', 'B-CPE-110_organized_virtualMark',
       'B-CPE-110_secured_Algorithm app. - Hash function_passed',
       'B-CPE-110_secured_Algorithm app. - Hash table_passed',
       'B-CPE-110_secured_Basics_passed',
       'B-CPE-110_secured_Data structure_passed',
       'B-CPE-110_secured_Optimization_passed',
       'B-CPE-110_secured_Robustness_passed', 'B-CPE-110_secured_mark',
       'B-CPE-110_secured_prerequisitesMark', 'B-CPE-110_secured_styleFatal',
       'B-CPE-110_secured_styleInfo', 'B

In [88]:
[c for c in cols_only_df1 if c.startswith('B-CPE-100')]

[]

In [89]:
common_cols = df22.columns.intersection(df23.columns)
common_cols.shape

(401,)

In [90]:
nb_nan_par_ligne = df22.isna().sum(axis=1)
# df22 = df22[nb_nan_par_ligne < 356]
nb_nan_par_ligne.sort_values().tail(10)

464     286
1122    286
227     302
422     312
1297    356
593     378
786     378
255     378
117     378
1119    382
dtype: int64

In [91]:
nb_nan_par_ligne = df23.isna().sum(axis=1)
# df23 = df23[nb_nan_par_ligne < 356]
print(df23.shape, nb_nan_par_ligne.sort_values().tail(20))

(1320, 486) 594     371
1125    371
224     371
1296    384
426     391
1056    415
1083    415
640     415
695     439
228     441
577     441
1264    441
881     441
861     441
1111    459
1131    461
614     461
31      463
529     463
834     463
dtype: int64


In [92]:
nb_nan_par_ligne = df23.isna().sum(axis=1)
# df22 = df22[nb_nan_par_ligne < 356]
nb_nan_par_ligne.sort_values().tail(10)

577     441
1264    441
881     441
861     441
1111    459
1131    461
614     461
31      463
529     463
834     463
dtype: int64